<a href="https://colab.research.google.com/github/rishi-tha16/DL-assignment2/blob/main/DL_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#Question2
import os
os.environ["WANDB_DISABLED"] = "true"  # Prevents W&B from asking for API key

# === STEP 1: Import Libraries ===
import pandas as pd
from datasets import load_dataset
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline
)

# === STEP 2: Load Excel and Extract Lyrics Column ===
df = pd.read_excel("/content/sample_data/Backstreet_Boys_Lyrics_score.xlsx ")

# Auto-detect column with 'lyric' in the name
lyrics_column = None
for col in df.columns:
    if "lyric" in col.lower():
        lyrics_column = col
        break

if not lyrics_column:
    raise ValueError("Could not find a lyrics column in the Excel file.")

# Join lyrics into one big string
lyrics_text = "\n\n".join(df[lyrics_column].dropna().astype(str).tolist())

# Save to plain text file
with open("lyrics.txt", "w", encoding="utf-8") as f:
    f.write(lyrics_text)

# === STEP 3: Load Dataset and Tokenizer ===
dataset = load_dataset("text", data_files={"train": "lyrics.txt"})

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# === STEP 4: Load GPT-2 Model ===
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Support pad token

# === STEP 5: Training Arguments (No W&B logging) ===
training_args = TrainingArguments(
    output_dir="./gpt2-lyrics",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_total_limit=1,
    logging_steps=10,
    report_to="none"  # Don't report to any logger (W&B, TensorBoard, etc.)
)

# === STEP 6: Trainer ===
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
)

# === STEP 7: Train the Model ===
trainer.train()

# === STEP 8: Save Model and Tokenizer ===
trainer.save_model("./gpt2-lyrics")
tokenizer.save_pretrained("./gpt2-lyrics")

# === STEP 9: Generate Sample Lyrics ===
generator = pipeline("text-generation", model="./gpt2-lyrics", tokenizer=tokenizer)

prompt = "I want it that way"
outputs = generator(prompt, max_length=100, num_return_sequences=1)

print("\n🎤 Generated Lyrics:\n")
print(outputs[0]["generated_text"])


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/6890 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.850100
20,3.764000
30,2.946500
40,3.615200
50,3.318700
60,3.749300
70,3.210100
80,3.710600
90,2.749300
100,3.227800


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



🎤 Generated Lyrics:

I want it that way (Yeah) I want it that way (Yeah) ive got it (Baby ive got it) ive got it (Yeah) ive gotta get it goin' on ive got it goin' on ive got it goin' on ive got it goin' on ive got it goin' on ike it goin' on ike it goin' on ike it goin' on ike it go


In [35]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# === STEP 1: Load and preprocess data ===
def load_data(path):
    df = pd.read_csv(path, sep='\t', header=None)
    df.dropna(inplace=True)
    return list(zip(df[1], df[0]))  # Latin, Devanagari

train_pairs = load_data('/content/hi.translit.sampled.train.tsv')
val_pairs = load_data('/content/hi.translit.sampled.train.tsv')

input_texts = [inp for inp, _ in train_pairs]
target_texts = ['\t' + tgt + '\n' for _, tgt in train_pairs]

input_chars = sorted(set(''.join(input_texts)))
target_chars = sorted(set(''.join(target_texts)))

input_token_index = {char: i for i, char in enumerate(input_chars)}
target_token_index = {char: i for i, char in enumerate(target_chars)}

reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens))
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens))
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens))

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# === STEP 2: Build Model ===
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64, epochs=10, validation_split=0.2)

# === STEP 3: Inference Models ===
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# === STEP 4: Decode Sequence ===
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    decoded_sentence = ''

    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n':
            break

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

    return decoded_sentence.strip()

# === STEP 5: Interactive Prediction ===
def encode_input_text(input_text):
    encoder_input = np.zeros((1, max_encoder_seq_length, num_encoder_tokens))
    for t, char in enumerate(input_text):
        if char in input_token_index:
            encoder_input[0, t, input_token_index[char]] = 1.
    return encoder_input

while True:
    user_input = input("Enter a Latin word (or type 'exit' to quit): ").strip().lower()
    if user_input == 'exit':
        break
    input_seq = encode_input_text(user_input)
    decoded = decode_sequence(input_seq)
    print(f"Predicted Devanagari: {decoded}")


Epoch 1/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.0634 - loss: 1.1846 - val_accuracy: 0.0625 - val_loss: 1.2827
Epoch 2/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0782 - loss: 1.1085 - val_accuracy: 0.0614 - val_loss: 1.3023
Epoch 3/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0926 - loss: 1.0583 - val_accuracy: 0.0594 - val_loss: 1.3149
Epoch 4/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1011 - loss: 1.0274 - val_accuracy: 0.0601 - val_loss: 1.3195
Epoch 5/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1061 - loss: 1.0073 - val_accuracy: 0.0535 - val_loss: 1.3235
Epoch 6/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1098 - loss: 0.9942 - val_accuracy: 0.0584 - val_loss: 1.3265
Epoch 7/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1129 - loss: 0.9844 - val_accuracy: 0.0597 - val_loss: 1.3256
Epoch 8/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1165 - loss: 0.9751 - val_accuracy: 0

In [37]:
#Question2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# === STEP 1: Load and preprocess data ===
def load_data(path):
    df = pd.read_csv(path, sep='\t', header=None)
    df.dropna(inplace=True)
    return list(zip(df[1], df[0]))  # Latin, Devanagari

train_pairs = load_data('/content/hi.translit.sampled.train.tsv')
val_pairs = load_data('/content/hi.translit.sampled.train.tsv')

input_texts = [inp for inp, _ in train_pairs]
target_texts = ['\t' + tgt + '\n' for _, tgt in train_pairs]

input_chars = sorted(set(''.join(input_texts)))
target_chars = sorted(set(''.join(target_texts)))

input_token_index = {char: i for i, char in enumerate(input_chars)}
target_token_index = {char: i for i, char in enumerate(target_chars)}

reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens))
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens))
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens))

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# === STEP 2: Build Model ===
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64, epochs=10, validation_split=0.2)

# === STEP 3: Inference Models ===
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# === STEP 4: Decode Sequence ===
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    decoded_sentence = ''

    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n':
            break

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

    return decoded_sentence.strip()

# === STEP 5: Interactive Prediction ===
def encode_input_text(input_text):
    encoder_input = np.zeros((1, max_encoder_seq_length, num_encoder_tokens))
    for t, char in enumerate(input_text):
        if char in input_token_index:
            encoder_input[0, t, input_token_index[char]] = 1.
    return encoder_input

while True:
    user_input = input("Enter a Latin word (or type 'exit' to quit): ").strip().lower()
    if user_input == 'exit':
        break
    input_seq = encode_input_text(user_input)
    decoded = decode_sequence(input_seq)
    print(f"Predicted Devanagari: {decoded}")


Epoch 1/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.0618 - loss: 1.1853 - val_accuracy: 0.0613 - val_loss: 1.2963
Epoch 2/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0755 - loss: 1.1037 - val_accuracy: 0.0626 - val_loss: 1.3087
Epoch 3/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0922 - loss: 1.0513 - val_accuracy: 0.0584 - val_loss: 1.3343
Epoch 4/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.1029 - loss: 1.0178 - val_accuracy: 0.0591 - val_loss: 1.3533
Epoch 5/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1105 - loss: 0.9940 - val_accuracy: 0.0620 - val_loss: 1.3581
Epoch 6/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1155 - loss: 0.9787 - val_accuracy: 0.0647 - val_loss: 1.3488
Epoch 7/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1193 - loss: 0.9667 - val_accuracy: 0.0648 - val_loss: 1.3421
Epoch 8/10
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1224 - loss: 0.9560 - val_accuracy: 0